In [1]:
# So when making the plan, the agent should first determine the broad sketches
# of the day, then fill in the details, and finally create the calendar events.
# in creating the calendar events, it should make sure to place them relativily,
# put some breathing room between them, and then generate the individual items 
'''
<calendarEvents>
  <!-- Morning Routine -->
  <calendarEvent>
    <title>Wake Up</title>
    <dtstart>2025-07-22T07:30:00</dtstart>
    <dtend>2025-07-22T07:30:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Morning Routine (Breakfast, Prep)</title>
    <dtstart>2025-07-22T07:30:00</dtstart>
    <dtend>2025-07-22T08:30:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Cycle to Office</title>
    <dtstart>2025-07-22T08:30:00</dtstart>
    <dtend>2025-07-22T09:00:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Day-Ahead Review</title>
    <dtstart>2025-07-22T09:00:00</dtstart>
    <dtend>2025-07-22T09:30:00</dtend>
  </calendarEvent>

  <!-- First Deep Work & Break -->
  <calendarEvent>
    <title>DeepWork: C2T – Partitional Dissonance</title>
    <dtstart>2025-07-22T09:30:00</dtstart>
    <dtend>2025-07-22T11:30:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Rejuvenation Break (Sci-Fi Reading)</title>
    <dtstart>2025-07-22T11:30:00</dtstart>
    <dtend>2025-07-22T12:00:00</dtend>
  </calendarEvent>

  <!-- FateForger Deep Work (with Lunch) -->
  <calendarEvent>
    <title>DeepWork: FateForger – Calendar Agent</title>
    <dtstart>2025-07-22T12:00:00</dtstart>
    <dtend>2025-07-22T14:00:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Lunch</title>
    <dtstart>2025-07-22T12:30:00</dtstart>
    <dtend>2025-07-22T13:00:00</dtend>
  </calendarEvent>

  <!-- Pre-Second Block Prep -->
  <calendarEvent>
    <title>Meditation (10 min)</title>
    <dtstart>2025-07-22T14:00:00</dtstart>
    <dtend>2025-07-22T14:10:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Buffer / Light Tasks</title>
    <dtstart>2025-07-22T14:10:00</dtstart>
    <dtend>2025-07-22T15:00:00</dtend>
  </calendarEvent>

  <!-- Second Deep Work -->
  <calendarEvent>
    <title>DeepWork: C2T – Gerimedica Verslaglegging</title>
    <dtstart>2025-07-22T15:00:00</dtstart>
    <dtend>2025-07-22T17:00:00</dtend>
  </calendarEvent>

  <!-- Quick Shutdown Rituals -->
  <calendarEvent>
    <title>TickTick Deep Clean</title>
    <dtstart>2025-07-22T17:00:00</dtstart>
    <dtend>2025-07-22T17:15:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Planning Session (Next Day)</title>
    <dtstart>2025-07-22T17:15:00</dtstart>
    <dtend>2025-07-22T17:30:00</dtend>
  </calendarEvent>

  <!-- Gym & Commute -->
  <calendarEvent>
    <title>Gym</title>
    <dtstart>2025-07-22T17:30:00</dtstart>
    <dtend>2025-07-22T19:00:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Cycle Home</title>
    <dtstart>2025-07-22T19:00:00</dtstart>
    <dtend>2025-07-22T19:30:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Log Workout in Strong App</title>
    <dtstart>2025-07-22T19:30:00</dtstart>
    <dtend>2025-07-22T19:40:00</dtend>
  </calendarEvent>

  <!-- Evening Shutdown Ritual Subtasks -->
  <calendarEvent>
    <title>Dog Walk</title>
    <dtstart>2025-07-22T22:00:00</dtstart>
    <dtend>2025-07-22T22:10:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Pack Bag &amp; Clothes</title>
    <dtstart>2025-07-22T22:10:00</dtstart>
    <dtend>2025-07-22T22:15:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Clean Face &amp; Brush Teeth</title>
    <dtstart>2025-07-22T22:15:00</dtstart>
    <dtend>2025-07-22T22:25:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Evening Reading</title>
    <dtstart>2025-07-22T22:25:00</dtstart>
    <dtend>2025-07-22T23:00:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Sleep</title>
    <dtstart>2025-07-22T23:00:00</dtstart>
    <dtend>2025-07-23T07:30:00</dtend>
  </calendarEvent>
</calendarEvents>
'''

'\n<calendarEvents>\n  <!-- Morning Routine -->\n  <calendarEvent>\n    <title>Wake Up</title>\n    <dtstart>2025-07-22T07:30:00</dtstart>\n    <dtend>2025-07-22T07:30:00</dtend>\n  </calendarEvent>\n  <calendarEvent>\n    <title>Morning Routine (Breakfast, Prep)</title>\n    <dtstart>2025-07-22T07:30:00</dtstart>\n    <dtend>2025-07-22T08:30:00</dtend>\n  </calendarEvent>\n  <calendarEvent>\n    <title>Cycle to Office</title>\n    <dtstart>2025-07-22T08:30:00</dtstart>\n    <dtend>2025-07-22T09:00:00</dtend>\n  </calendarEvent>\n  <calendarEvent>\n    <title>Day-Ahead Review</title>\n    <dtstart>2025-07-22T09:00:00</dtstart>\n    <dtend>2025-07-22T09:30:00</dtend>\n  </calendarEvent>\n\n  <!-- First Deep Work & Break -->\n  <calendarEvent>\n    <title>DeepWork: C2T – Partitional Dissonance</title>\n    <dtstart>2025-07-22T09:30:00</dtstart>\n    <dtend>2025-07-22T11:30:00</dtend>\n  </calendarEvent>\n  <calendarEvent>\n    <title>Rejuvenation Break (Sci-Fi Reading)</title>\n    <dtst

In [1]:
import logging
import sys

# 1) Reset any existing handlers and force new config (Python ≥3.8)
logging.root.handlers.clear()
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(name)s %(levelname)s %(message)s",
    force=True,           # override any prior config
    stream=sys.stdout,
)

# 2) Names from the logging docs
from autogen_core import TRACE_LOGGER_NAME as CORE_TRACE, EVENT_LOGGER_NAME as CORE_EVENT
from autogen_agentchat import TRACE_LOGGER_NAME as CHAT_TRACE, EVENT_LOGGER_NAME as CHAT_EVENT

# 3) Explicitly enable each
for logger_name in (
    CORE_TRACE,
    CORE_EVENT,
    CHAT_TRACE,
    CHAT_EVENT,
):
    lg = logging.getLogger(logger_name)
    lg.setLevel(logging.DEBUG)
    if not lg.handlers:
        lg.addHandler(logging.StreamHandler(sys.stdout))

# 4) Finally, also turn on any “root” AutoGen logs not covered above
logging.getLogger("autogen_core").setLevel(logging.DEBUG)
logging.getLogger("autogen_agentchat").setLevel(logging.DEBUG)

In [2]:
# Force reload all modules to get the latest agent code
import sys
import importlib

# Clear ALL fateforger modules from cache
modules_to_clear = [name for name in sys.modules.keys() if name.startswith('fateforger') or name.startswith('tools_config')]
for module_name in modules_to_clear:
    del sys.modules[module_name]
    print(f"Cleared module: {module_name}")

from fateforger.core.runtime import initialize_runtime
from fateforger.core.config import settings
from autogen_core import AgentId
import asyncio

print("Initializing runtime with fresh agent code...")

runtime = None
try:
    # Initialize runtime with fresh agents
    runtime = await asyncio.wait_for(initialize_runtime(), timeout=10.0)

    runtime.start()
    print("Runtime initialized successfully!")
    
    print("Sending message to planner agent...")
    
    # Import Message after runtime is initialized to avoid import issues
    from fateforger.agents.schedular.agent import Message
    
    # Send message and await response
    response = await asyncio.wait_for(
        runtime.send_message(
            Message(content="What meetings do I have tomorrow?"),
            recipient=AgentId("planner_agent", "default")
        ),
        timeout=15.0
    )
    
    print(f"Response received: {response}")
    print(f"Response type: {type(response)}")
    if hasattr(response, 'content'):
        print(f"Response content: {response.content}")
        
except asyncio.TimeoutError:
    print("TIMEOUT: Agent took too long to respond!")
except Exception as e:
    print(f"Error occurred: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Clean shutdown
    if runtime:
        try:
            await runtime.stop()
            print("Runtime stopped cleanly.")
        except Exception as e:
            print(f"Error stopping runtime: {e}")

print("Test complete!")

ImportError: cannot import name 'CalendarEvent' from 'fateforger.agents.schedular.models' (/Users/hugoevers/VScode-projects/admonish-1/src/fateforger/agents/schedular/models/__init__.py)

In [3]:
from fateforger.core.runtime import initialize_runtime
from autogen_core import AgentId
import asyncio
from fateforger.agents.schedular.agent import MyMessage


runtime = await asyncio.wait_for(initialize_runtime(), timeout=10.0)
runtime.start()

response = await asyncio.wait_for(
    runtime.send_message(
        MyMessage(content="When is my first event today?"),
        recipient=AgentId("planner_agent", "default")
    ),
    timeout=15.0
)

if hasattr(response, 'content'):
    print(f"Response content: {response.content}")


ImportError: cannot import name 'CalendarEvent' from 'fateforger.agents.schedular.models' (/Users/hugoevers/VScode-projects/admonish-1/src/fateforger/agents/schedular/models/__init__.py)

In [2]:
response

TextMessage(id='ef4c229d-9c6a-4fef-9f7e-0cb08e354ca6', source='planner_agent', models_usage=RequestUsage(prompt_tokens=254, completion_tokens=22), metadata={}, created_at=datetime.datetime(2025, 7, 27, 21, 47, 19, 443537, tzinfo=datetime.timezone.utc), content="Let's check your primary calendar for any events on 2025-07-28. One moment, please.", type='TextMessage')

In [1]:
from fateforger.core.bootstrap import runtime
from fateforger.models.events import CalendarEvent
from autogen_core import AgentId

# all agents should be pre-registered
runtime.start()

await runtime.send_message(
    CalendarEvent(summary="Demo", start={...}, end={...}),
    recipient=AgentId("planner","default")      # LLM decides tool
)
# how do i get the message out?
await asyncio.sleep(1)
await runtime.stop()

ImportError: cannot import name 'runtime' from 'fateforger.core.bootstrap' (/Users/hugoevers/VScode-projects/admonish-1/src/fateforger/core/bootstrap.py)

In [ ]:
# TODO: add the slack send tool to the chain later